# DNN Model Notebook

```md
@authors: miguelrocha and Grupo 03
```

In [22]:
# Notebook Imports
import numpy as np

from models.dnn_model import NeuralNetwork, hyperparameter_optimization
from helpers.dataset import Dataset
from helpers.enums import ModelRunMode
from helpers.layers import DenseLayer
from helpers.activation import ReLUActivation, SigmoidActivation
from helpers.losses import BinaryCrossEntropy
from helpers.optimizer import Optimizer
from helpers.metrics import accuracy
from helpers.enums import ModelType
from helpers.model import save_dnn_model

In [23]:
# Model run mode
# Options: 
#   ModelRunMode.TRAIN.value            (Train the model)
#   ModelRunMode.CLASSIFY.value         (Classify data)
mode = ModelRunMode.TRAIN.value
# Prefix for saving the model files
model_prefix = "dnn_model_1"

In [24]:
# Parameters cell
if mode == ModelRunMode.TRAIN.value:
    # TRAIN mode: Set parameters for training
    input_csv = "../tarefa_1/clean_input_datasets/dataset1_enh_inputs.csv"              # CSV file with training inputs (ID, Text)
    output_csv = "../tarefa_1/clean_output_datasets/dataset1_enh_outputs.csv"           # CSV file with training outputs (ID, Label)
    test_size = 0.3                                                                     # Proportion of the dataset to use as test data
    validation_size = 0.2                                                               # Proportion of the dataset reserved as validation data (used during hyperparameter tuning)
    verbose = True                                                                      # Verbosity level enabler
    random_state=42                                                                     # Seed for reproducible dataset splitting
    max_vocab_size=None                                                                 # Maximum vocabulary size (None implies no limit)
    min_freq=1                                                                          # Minimum frequency required for a word to be included in the vocabulary
elif mode == ModelRunMode.CLASSIFY.value:
    # CLASSIFY mode: Set parameters for classification
    input_csv = "../tarefa_2/classify_input_datasets/dataset2_inputs.csv"               # CSV file with texts for prediction (ID, Text)
    output_csv = "../tarefa_2/classify_output_datasets/dataset2_outputs_dnn_model.csv"  # CSV file to store prediction result
else:
    print("The selected option is not valid. Options: \"train\" or \"classify\"!")
    SystemExit()

In [25]:
if mode == ModelRunMode.TRAIN.value:
    # Load datasets using TF-IDF vectorization
    X_train, y_train, X_test, y_test, vocab, idf = Dataset.prepare_train_test_tfidf(input_csv=input_csv, output_csv=output_csv, test_size=test_size, random_state=random_state, max_vocab_size=max_vocab_size, min_freq=min_freq)

    # Create Dataset objects for training and test data
    train_ds_full = Dataset(X=X_train, Y=y_train)
    test_ds = Dataset(X=X_test, Y=y_test)

    # Display dimensions of the training and test datasets
    print(f"Train set has {train_ds_full.nrows()} rows and {train_ds_full.ncols()} columns")
    print(f"Test set has {test_ds.nrows()} rows and {test_ds.ncols()} columns\n")

    # Split the full training dataset into training and validation subsets (80% training, 20% validation)
    n_train = train_ds_full.X.shape[0]
    indices = np.arange(n_train)
    np.random.shuffle(indices)
    split_idx = int((1 - validation_size) * n_train)
    train_idx = indices[:split_idx]
    val_idx = indices[split_idx:]
    train_ds = Dataset(X=train_ds_full.X[train_idx], Y=train_ds_full.Y[train_idx])
    val_ds = Dataset(X=train_ds_full.X[val_idx], Y=train_ds_full.Y[val_idx])

    # More varied epochs to handle both simpler and more complex tasks
    epochs_list = [40, 60, 80, 100, 120, 140]

    # A wider range of batch sizes
    batch_size_list = [8, 16, 32]

    # Learning rate from very small to moderate
    learning_rate_list = [0.001, 0.01, 0.1]

    # Momentum slightly varied around typical defaults
    momentum_list = [0.85, 0.9, 0.95]

    # Different hidden layer topologies
    hidden_layers_list = [
        [32],
        [64],
        [64, 32],
        [128, 64]
    ]

    # Add mild to moderate dropout levels
    dropout_list = [0.0, 0.01, 0.1, 0.5]

    # Perform hyperparameter tuning using the training and validation sets
    print("Starting hyperparameter tuning...")
    best_params = hyperparameter_optimization(train_ds, val_ds, epochs_list, batch_size_list, learning_rate_list, momentum_list, hidden_layers_list, dropout_list, n_iter=20)
    print("\nBest hyperparameters:", best_params)

    # Retrain DNN model on the full training data using the best hyperparameters
    final_model = NeuralNetwork(epochs=best_params['epochs'], batch_size=best_params['batch_size'], optimizer=Optimizer(learning_rate=best_params['learning_rate'], momentum=best_params['momentum']), verbose=verbose, loss=BinaryCrossEntropy,  metric=accuracy)

    # Build model Layers
    n_features = train_ds.X.shape[1]
    for i, units in enumerate(best_params['n_hidden']):
        if i == 0:
            final_model.add(DenseLayer(units, (n_features,),dropout_rate=best_params['dropout_rate']))
        else:
            final_model.add(DenseLayer(units,dropout_rate=best_params['dropout_rate']))
        final_model.add(ReLUActivation())
        
    final_model.add(DenseLayer(1))
    final_model.add(SigmoidActivation())

    # Fit model
    final_model.fit(train_ds)
    out = final_model.predict(test_ds)

    # Evaluate the final model on the test set
    acc = final_model.score(test_ds, out)
    print("Test Accuracy:", acc)

    # Save the model, plus vocab & idf, so classification can replicate
    save_dnn_model(dnn=final_model, vocab=vocab, idf=idf, model_prefix=model_prefix)
    print(f"Model saved with prefix {model_prefix}")

if mode == ModelRunMode.CLASSIFY.value:
    # Classify new texts using the saved model
    Dataset.classify_texts(input_csv=input_csv, output_csv=output_csv, neural_net_class=NeuralNetwork, model_type=ModelType.DNN ,model_prefix=model_prefix)

Train set has 70 rows and 2124 columns
Test set has 30 rows and 2124 columns

Starting hyperparameter tuning...


Hyperparameter Search:   5%|▌         | 1/20 [00:00<00:04,  4.69it/s]

Parameters: {'epochs': 100, 'batch_size': 16, 'learning_rate': 0.01, 'momentum': 0.95, 'n_hidden': [32], 'dropout_rate': 0.0}  |  Accuracy: 0.4286


Hyperparameter Search:  10%|█         | 2/20 [00:00<00:09,  1.93it/s]

Parameters: {'epochs': 100, 'batch_size': 16, 'learning_rate': 0.001, 'momentum': 0.85, 'n_hidden': [64, 32], 'dropout_rate': 0.1}  |  Accuracy: 0.3571


...

Hyperparameter Search:  95%|█████████▌| 19/20 [00:15<00:01,  1.28s/it]

Parameters: {'epochs': 140, 'batch_size': 16, 'learning_rate': 0.001, 'momentum': 0.85, 'n_hidden': [128, 64], 'dropout_rate': 0.01}  |  Accuracy: 0.5000


Hyperparameter Search: 100%|██████████| 20/20 [00:17<00:00,  1.17it/s]

Parameters: {'epochs': 100, 'batch_size': 8, 'learning_rate': 0.001, 'momentum': 0.95, 'n_hidden': [64], 'dropout_rate': 0.01}  |  Accuracy: 0.4286

Best Hyperparameters Found: {'epochs': 40, 'batch_size': 16, 'learning_rate': 0.1, 'momentum': 0.95, 'n_hidden': [64], 'dropout_rate': 0.0}
Best Accuracy: 0.8571

Best hyperparameters: {'epochs': 40, 'batch_size': 16, 'learning_rate': 0.1, 'momentum': 0.95, 'n_hidden': [64], 'dropout_rate': 0.0}
Epoch 1/40 - loss: 0.6650 - accuracy: 0.6250
Epoch 2/40 - loss: 0.6664 - accuracy: 0.6250
Epoch 3/40 - loss: 0.6475 - accuracy: 0.6250
Epoch 4/40 - loss: 0.6304 - accuracy: 0.7292
Epoch 5/40 - loss: 0.6162 - accuracy: 0.7500
Epoch 6/40 - loss: 0.6026 - accuracy: 0.8125
Epoch 7/40 - loss: 0.5748 - accuracy: 0.8750
Epoch 8/40 - loss: 0.5737 - accuracy: 0.8542
Epoch 9/40 - loss: 0.5402 - accuracy: 0.8542
Epoch 10/40 - loss: 0.5123 - accuracy: 0.8750
Epoch 11/40 - loss: 0.4873 - accuracy: 0.8750
Epoch 12/40 - loss: 0.4608 - accuracy: 0.9167
Epoch 13/40

Epoch 31/40 - loss: 0.0412 - accuracy: 1.0000
Epoch 32/40 - loss: 0.0354 - accuracy: 1.0000
Epoch 33/40 - loss: 0.0308 - accuracy: 1.0000
Epoch 34/40 - loss: 0.0305 - accuracy: 1.0000
Epoch 35/40 - loss: 0.0255 - accuracy: 1.0000
Epoch 36/40 - loss: 0.0234 - accuracy: 1.0000
Epoch 37/40 - loss: 0.0222 - accuracy: 1.0000
Epoch 38/40 - loss: 0.0194 - accuracy: 1.0000
Epoch 39/40 - loss: 0.0159 - accuracy: 1.0000
Epoch 40/40 - loss: 0.0171 - accuracy: 1.0000
Test Accuracy: 0.36666666666666664
DNN Model saved to folder 'dnn_model_weights'. JSON = 'dnn_model_weights\dnn_model_1_architecture.json'
Model saved with prefix dnn_model_1
